### GLM-eb 
@author: Jordan, Ben

#### setup:

In [1]:
# preamble
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pandas as pd
import scipy.stats as stats
import scipy as sp
import statistics

In [2]:
import glmmod

In [3]:
# visualization parameters
plt.rcParams['figure.figsize'] = (4,2)
plt.rc('axes', labelsize=10); plt.rc('axes', titlesize=10)
plt.style.use('ggplot'); plt.rc('font', size=10);

In [4]:
# load & format sample data
filepath = 'sampleData.mat'
mat = scipy.io.loadmat(filepath)
ST = mat['ST']; P = mat['P']; hd = mat['hd']

#### run glm:

In [232]:
# initialize class instance
g = glmmod.glm(ST,P,hd)

# prepare the data
posgrid_raw,bins = g.pos_map(nbins=10)
ebgrid_raw,bins = g.eb_map(nbins=10, rp=[75,75])
smooth_fr, raw_spktrn, filt, dt = g.conv_spktrain() # get spiketrain
posgrid,ebgrid,spiketrain = g.speed_threshold(posgrid_raw,ebgrid_raw,raw_spktrn)

# dictionaries with info about each model
stateDict = {
    0: [posgrid,ebgrid],
    1: posgrid,
    2: ebgrid
}

labelDict = {
    0: 'PE',
    1: 'P',
    2: 'E'
}

allModels = {}
numModels = 3

# get test/train indices (same for each model)
kfoldIdx, kfoldIdx_df = g.kfoldSplit(nfolds=10)

for model in range(numModels):
    modelDict = {}
    # get state matrix
    stateIn = stateDict[model]
    statemat, expr = g.squish_statemat(spiketrain, stateIn, modelType=labelDict[model])

    # optimize model parameters
    kres,train_y, test_y, train_x, test_x, train_y_raw, test_y_raw= g.kfoldOptim(kfoldIdx_df,statemat)

    # check the model fit
    testfit = g.get_testFit(kres,train_y,test_y,train_x,test_x,train_y_raw,test_y_raw)

    modelDict['kfoldIdx'] = kfoldIdx_df
    modelDict['kres'] = kres
    modelDict['train_y'] = train_y
    modelDict['train_x'] = train_x
    modelDict['test_y'] = test_y
    modelDict['test_x'] = test_x
    modelDict['train_y_raw'] = train_y_raw
    modelDict['test_y_raw'] = test_y_raw
    modelDict['testfit'] = testfit
    
    # save in allModels dictionary
    allModels[model] = modelDict

llh, bestModel = g.findBestModel(allModels)

g.plot_llh(allModels,labelDict)

NameError: name 'allModels' is not defined

### compute penalization terms

In [228]:
beta = 5e1 # for example
XX = allModels[1]['test_x'][0] # state matrix
_,numvar = np.shape(XX)
init_param = 1e-3*np.random.randn(numvar) # from random normal distrib
param = init_param

In [230]:
J, J_g, J_h = rough_penalty(param,beta,vartype='1D-circ')

In [ ]:
# for penalization stuff
if smooth:
    fpen,fgrad = penalize(params,X,spike_train,smoothers)
    f += fpen
    grad += fgrad